In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [48]:
# Settings
methods = ["ele","da","jb","sig","esig"]
method = "ele"
eBeam = 10
pBeam = 250
beamConfig = f"{eBeam}x{pBeam}"
binning = 'HeraBinning' # 'StandardBinning' or 'HeraBinning' 
lumi = 1. # target luminosity in fb-1 (requires that EvalYields.ipynb be run with this target lumi)
pdfset = "NNPDF31_nnlo_as_0118_proton"

In [49]:
# SETTINGS FOR SYSTEMATICS
# YR inspired
point_to_point_err_low_percent = 1.2
point_to_point_err_high_percent = 4.2
normalisation_err_low_percent = 2.5
normalisation_err_high_percent = 4.3
# HERA inspired
# point_to_point_err_low_percent = 1.
# point_to_point_err_high_percent = 1.9 
# normalisation_err_low_percent = 0.
# normalisation_err_high_percent = 3.4
y_degradation_threshold = 0.01 # Threshold value below which systematics are degraded as 1 / y (0.1 or 0.05 for now) (0.01 to avoid)

In [50]:
binning_df = pd.read_csv(f"../Tables/{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_bins.csv")
binning_df

,x,Q2,y,gen,rec,gen_rec,purity,stability
0,0.00032,2.0,0.625000,7.152524e+06,3.919886e+06,3.384904e+06,0.863521,0.473246
1,0.00050,2.0,0.400000,7.949139e+06,6.313385e+06,4.982386e+06,0.789178,0.626783
2,0.00080,2.0,0.250000,8.768433e+06,8.484940e+06,5.726555e+06,0.674908,0.653088
3,0.00100,2.0,0.200000,8.571405e+06,9.763493e+06,4.749922e+06,0.486498,0.554159
4,0.00200,2.0,0.100000,8.509037e+06,1.031914e+07,3.383487e+06,0.327885,0.397635
...,...,...,...,...,...,...,...,...
274,0.40000,2000.0,0.500000,1.329739e+03,6.722174e+02,5.138245e+02,0.764372,0.386410
275,0.65000,2000.0,0.307692,3.042668e+02,2.307856e+02,1.328106e+02,0.575472,0.436494
276,0.40000,3000.0,0.750000,4.849763e+02,1.660132e+02,1.338992e+02,0.806557,0.276094
277,0.65000,3000.0,0.461538,1.192029e+02,6.096223e+01,4.626598e+01,0.758929,0.388128


In [51]:
xsec_df = pd.read_csv(f"../Tables/{binning}_{method}_Ee{eBeam}_Ep{pBeam}_{pdfset}_XsecPredictions.csv")
xsec_df

,Q2,x,y,sigma_red_0,sigma_red_unc
0,2.0,0.00032,0.625000,0.452226,0.034219
1,2.0,0.00050,0.400000,0.419979,0.022402
2,2.0,0.00080,0.250000,0.395403,0.018818
3,2.0,0.00100,0.200000,0.384916,0.017994
4,2.0,0.00200,0.100000,0.349494,0.011605
...,...,...,...,...,...
274,2000.0,0.40000,0.500000,0.137081,0.001838
275,2000.0,0.65000,0.307692,0.019099,0.000804
276,3000.0,0.40000,0.750000,0.148814,0.002374
277,3000.0,0.65000,0.461538,0.019774,0.001203


In [52]:
# Tolerances for dataframe merging
tol_x  = 1e-5
tol_Q2 = 1e-3
tol_y  = 1e-4

def merge_dataframes(binning_df, xsec_df,
                   tol_x=1e-6, tol_Q2=1e-6, tol_y=1e-6):
    """
    Merge binning and xsec dataframes on (Q2, x, y) values, within tolerances.
    Keep only rows with a matching xsec_df entry.
    """

    # sort xsec_df - speeds up search
    xsec_sorted = xsec_df.sort_values(["Q2", "x"]).reset_index(drop=True)
    # xsec_sorted = xsec_df

    matched_rows = []
    unmatched_count = 0

    for _, row in binning_df.iterrows():
        # find candidate matches within tolerance
        candidates = xsec_sorted[
            (abs(xsec_sorted["Q2"] - row["Q2"]) < tol_Q2) &
            (abs(xsec_sorted["x"]  - row["x"])  < tol_x)  &
            (abs(xsec_sorted["y"]  - row["y"])  < tol_y)
        ]

        if len(candidates) == 0:
            unmatched_count += 1
            continue   # drop row (no match)

        # If multiple matches exist, take the closest one
        cand = candidates.iloc[0]

        merged_row = {
            "Q2": row["Q2"],
            "x": row["x"],
            "y": row["y"],
            "gen": row["gen"],
            "rec": row["rec"],
            "gen_rec": row["gen_rec"],
            "purity": row["purity"],
            "stability": row["stability"],
            "sigma_red_0": cand["sigma_red_0"],
            "sigma_red_unc": cand["sigma_red_unc"],
        }

        matched_rows.append(merged_row)

    # Convert to dataframe
    merged_df = pd.DataFrame(matched_rows)

    # Print warning if rows were dropped
    if unmatched_count > 0:
        print(f"Warning: {unmatched_count} rows from binning_df had no matching xsec_df entry and were dropped.")

    return merged_df


merged_df = merge_dataframes(binning_df, xsec_df, tol_x, tol_Q2, tol_y)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red_0,sigma_red_unc
0,2.0,0.00032,0.625000,7.152524e+06,3.919886e+06,3.384904e+06,0.863521,0.473246,0.452226,0.034219
1,2.0,0.00050,0.400000,7.949139e+06,6.313385e+06,4.982386e+06,0.789178,0.626783,0.419979,0.022402
2,2.0,0.00080,0.250000,8.768433e+06,8.484940e+06,5.726555e+06,0.674908,0.653088,0.395403,0.018818
3,2.0,0.00100,0.200000,8.571405e+06,9.763493e+06,4.749922e+06,0.486498,0.554159,0.384916,0.017994
4,2.0,0.00200,0.100000,8.509037e+06,1.031914e+07,3.383487e+06,0.327885,0.397635,0.349494,0.011605
...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,1.329739e+03,6.722174e+02,5.138245e+02,0.764372,0.386410,0.137081,0.001838
275,2000.0,0.65000,0.307692,3.042668e+02,2.307856e+02,1.328106e+02,0.575472,0.436494,0.019099,0.000804
276,3000.0,0.40000,0.750000,4.849763e+02,1.660132e+02,1.338992e+02,0.806557,0.276094,0.148814,0.002374
277,3000.0,0.65000,0.461538,1.192029e+02,6.096223e+01,4.626598e+01,0.758929,0.388128,0.019774,0.001203


In [53]:
# Statistical uncertainties (the only thing coming from ePIC currently)

def calc_sigma_stat(row):
    """
    Get statistical uncertainty IN PERCENT
    """
    return 100/np.sqrt(row['rec'])

merged_df['sigma_stat'] = merged_df.apply(calc_sigma_stat, axis=1)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red_0,sigma_red_unc,sigma_stat
0,2.0,0.00032,0.625000,7.152524e+06,3.919886e+06,3.384904e+06,0.863521,0.473246,0.452226,0.034219,0.050508
1,2.0,0.00050,0.400000,7.949139e+06,6.313385e+06,4.982386e+06,0.789178,0.626783,0.419979,0.022402,0.039799
2,2.0,0.00080,0.250000,8.768433e+06,8.484940e+06,5.726555e+06,0.674908,0.653088,0.395403,0.018818,0.034330
3,2.0,0.00100,0.200000,8.571405e+06,9.763493e+06,4.749922e+06,0.486498,0.554159,0.384916,0.017994,0.032003
4,2.0,0.00200,0.100000,8.509037e+06,1.031914e+07,3.383487e+06,0.327885,0.397635,0.349494,0.011605,0.031130
...,...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,1.329739e+03,6.722174e+02,5.138245e+02,0.764372,0.386410,0.137081,0.001838,3.856960
275,2000.0,0.65000,0.307692,3.042668e+02,2.307856e+02,1.328106e+02,0.575472,0.436494,0.019099,0.000804,6.582573
276,3000.0,0.40000,0.750000,4.849763e+02,1.660132e+02,1.338992e+02,0.806557,0.276094,0.148814,0.002374,7.761197
277,3000.0,0.65000,0.461538,1.192029e+02,6.096223e+01,4.626598e+01,0.758929,0.388128,0.019774,0.001203,12.807654


In [54]:
sigma_p2p_opt_list = [point_to_point_err_low_percent]*len(merged_df.index)
sigma_p2p_pess_list = [point_to_point_err_high_percent]*len(merged_df.index)
sigma_norm_opt_list = [normalisation_err_low_percent]*len(merged_df.index)
sigma_norm_pess_list = [normalisation_err_high_percent]*len(merged_df.index)

merged_df["sigma_p2p_opt"] = sigma_p2p_opt_list
merged_df["sigma_p2p_pess"] = sigma_p2p_pess_list
merged_df["sigma_norm_opt"] = sigma_norm_opt_list
merged_df["sigma_norm_pess"] = sigma_norm_pess_list

merged_df = merged_df.rename(columns={"sigma_red_0": "sigma_red", "sigma_red_unc": "sigma_red_pdf_unc"}) # rename for convenience
#merged_df = merged_df.rename(columns={"sigma_red_unc": "sigma_red_pdf_unc"})

merged_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_all_columns_grid.csv",index=False)
merged_df

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p_opt,sigma_p2p_pess,sigma_norm_opt,sigma_norm_pess
0,2.0,0.00032,0.625000,7.152524e+06,3.919886e+06,3.384904e+06,0.863521,0.473246,0.452226,0.034219,0.050508,1.2,4.2,2.5,4.3
1,2.0,0.00050,0.400000,7.949139e+06,6.313385e+06,4.982386e+06,0.789178,0.626783,0.419979,0.022402,0.039799,1.2,4.2,2.5,4.3
2,2.0,0.00080,0.250000,8.768433e+06,8.484940e+06,5.726555e+06,0.674908,0.653088,0.395403,0.018818,0.034330,1.2,4.2,2.5,4.3
3,2.0,0.00100,0.200000,8.571405e+06,9.763493e+06,4.749922e+06,0.486498,0.554159,0.384916,0.017994,0.032003,1.2,4.2,2.5,4.3
4,2.0,0.00200,0.100000,8.509037e+06,1.031914e+07,3.383487e+06,0.327885,0.397635,0.349494,0.011605,0.031130,1.2,4.2,2.5,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,1.329739e+03,6.722174e+02,5.138245e+02,0.764372,0.386410,0.137081,0.001838,3.856960,1.2,4.2,2.5,4.3
275,2000.0,0.65000,0.307692,3.042668e+02,2.307856e+02,1.328106e+02,0.575472,0.436494,0.019099,0.000804,6.582573,1.2,4.2,2.5,4.3
276,3000.0,0.40000,0.750000,4.849763e+02,1.660132e+02,1.338992e+02,0.806557,0.276094,0.148814,0.002374,7.761197,1.2,4.2,2.5,4.3
277,3000.0,0.65000,0.461538,1.192029e+02,6.096223e+01,4.626598e+01,0.758929,0.388128,0.019774,0.001203,12.807654,1.2,4.2,2.5,4.3


In [55]:
# Write optimistic grid
opt_df = merged_df[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_opt','sigma_norm_opt']]
opt_df = opt_df.rename(columns={"sigma_p2p_opt":"sigma_p2p","sigma_norm_opt":"sigma_norm"})
opt_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_optimistic_grid.csv",index=False)
opt_df

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,2.0,0.00032,0.625000,0.863521,0.473246,0.452226,0.034219,0.050508,1.2,2.5
1,2.0,0.00050,0.400000,0.789178,0.626783,0.419979,0.022402,0.039799,1.2,2.5
2,2.0,0.00080,0.250000,0.674908,0.653088,0.395403,0.018818,0.034330,1.2,2.5
3,2.0,0.00100,0.200000,0.486498,0.554159,0.384916,0.017994,0.032003,1.2,2.5
4,2.0,0.00200,0.100000,0.327885,0.397635,0.349494,0.011605,0.031130,1.2,2.5
...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,0.764372,0.386410,0.137081,0.001838,3.856960,1.2,2.5
275,2000.0,0.65000,0.307692,0.575472,0.436494,0.019099,0.000804,6.582573,1.2,2.5
276,3000.0,0.40000,0.750000,0.806557,0.276094,0.148814,0.002374,7.761197,1.2,2.5
277,3000.0,0.65000,0.461538,0.758929,0.388128,0.019774,0.001203,12.807654,1.2,2.5


In [56]:
# Write pessimistic grid
pess_df = merged_df[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_pess','sigma_norm_pess']]
pess_df = pess_df.rename(columns={"sigma_p2p_pess":"sigma_p2p","sigma_norm_pess":"sigma_norm"})
pess_df.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_pessimistic_grid.csv",index=False)
pess_df

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,2.0,0.00032,0.625000,0.863521,0.473246,0.452226,0.034219,0.050508,4.2,4.3
1,2.0,0.00050,0.400000,0.789178,0.626783,0.419979,0.022402,0.039799,4.2,4.3
2,2.0,0.00080,0.250000,0.674908,0.653088,0.395403,0.018818,0.034330,4.2,4.3
3,2.0,0.00100,0.200000,0.486498,0.554159,0.384916,0.017994,0.032003,4.2,4.3
4,2.0,0.00200,0.100000,0.327885,0.397635,0.349494,0.011605,0.031130,4.2,4.3
...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,0.764372,0.386410,0.137081,0.001838,3.856960,4.2,4.3
275,2000.0,0.65000,0.307692,0.575472,0.436494,0.019099,0.000804,6.582573,4.2,4.3
276,3000.0,0.40000,0.750000,0.806557,0.276094,0.148814,0.002374,7.761197,4.2,4.3
277,3000.0,0.65000,0.461538,0.758929,0.388128,0.019774,0.001203,12.807654,4.2,4.3


In [57]:
# added this after the other degrade y function to keep rather than replace the existing sigma_p2p
def degrade_y_keep_column(row):
    """
    Degrade p2p systematics as 1/y below y_degradation_threshold
    """
    if row['y'] < y_degradation_threshold:
        row['sigma_p2p_pess'] *= (y_degradation_threshold/row['y'])
    return row

merged_df_y_degraded = merged_df.apply(degrade_y_keep_column, axis=1)

merged_df_y_degraded.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_all_columns_y_degraded_below_{y_degradation_threshold}_grid.csv",index=False)
merged_df_y_degraded

,Q2,x,y,gen,rec,gen_rec,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p_opt,sigma_p2p_pess,sigma_norm_opt,sigma_norm_pess
0,2.0,0.00032,0.625000,7.152524e+06,3.919886e+06,3.384904e+06,0.863521,0.473246,0.452226,0.034219,0.050508,1.2,4.2,2.5,4.3
1,2.0,0.00050,0.400000,7.949139e+06,6.313385e+06,4.982386e+06,0.789178,0.626783,0.419979,0.022402,0.039799,1.2,4.2,2.5,4.3
2,2.0,0.00080,0.250000,8.768433e+06,8.484940e+06,5.726555e+06,0.674908,0.653088,0.395403,0.018818,0.034330,1.2,4.2,2.5,4.3
3,2.0,0.00100,0.200000,8.571405e+06,9.763493e+06,4.749922e+06,0.486498,0.554159,0.384916,0.017994,0.032003,1.2,4.2,2.5,4.3
4,2.0,0.00200,0.100000,8.509037e+06,1.031914e+07,3.383487e+06,0.327885,0.397635,0.349494,0.011605,0.031130,1.2,4.2,2.5,4.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,2000.0,0.40000,0.500000,1.329739e+03,6.722174e+02,5.138245e+02,0.764372,0.386410,0.137081,0.001838,3.856960,1.2,4.2,2.5,4.3
275,2000.0,0.65000,0.307692,3.042668e+02,2.307856e+02,1.328106e+02,0.575472,0.436494,0.019099,0.000804,6.582573,1.2,4.2,2.5,4.3
276,3000.0,0.40000,0.750000,4.849763e+02,1.660132e+02,1.338992e+02,0.806557,0.276094,0.148814,0.002374,7.761197,1.2,4.2,2.5,4.3
277,3000.0,0.65000,0.461538,1.192029e+02,6.096223e+01,4.626598e+01,0.758929,0.388128,0.019774,0.001203,12.807654,1.2,4.2,2.5,4.3


In [23]:
def degrade_y(row):
    """
    Degrade p2p systematics as 1/y below y_degradation_threshold
    """
    if row['y'] < y_degradation_threshold:
        row['sigma_p2p'] *= (y_degradation_threshold/row['y'])
    return row

# Write pessimistic y degraded grid
# if method == "ele":
pess_df_y_degraded = merged_df_y_degraded[['Q2','x','y','purity','stability','sigma_red','sigma_red_pdf_unc','sigma_stat','sigma_p2p_pess','sigma_norm_pess']]
pess_df_y_degraded = pess_df_y_degraded.rename(columns={"sigma_p2p_pess":"sigma_p2p","sigma_norm_pess":"sigma_norm"})   
# degrade p2p systematics as 1/y
    # pess_df_y_degraded = pess_df.apply(degrade_y, axis=1)

pess_df_y_degraded.to_csv(f"../FinalGrids/{pdfset}_{binning}_{method}_Ee{eBeam}_Ep{pBeam}_lumi{lumi}fb-1_pessimistic_y_degraded_below_{y_degradation_threshold}_grid.csv",index=False)
pess_df_y_degraded

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,1.258925,0.000200,0.630957,0.869573,0.473880,0.320509,0.031323,0.033080,1.9,3.4
1,1.258925,0.000316,0.398107,0.778747,0.575700,0.290023,0.026500,0.026445,1.9,3.4
2,1.258925,0.000501,0.251189,0.590162,0.564724,0.270261,0.016928,0.022658,1.9,3.4
3,1.258925,0.000794,0.158489,0.382035,0.453197,0.254951,0.013751,0.020027,1.9,3.4
4,1.258925,0.001259,0.100000,0.227740,0.300048,0.240664,0.012491,0.019144,1.9,3.4
...,...,...,...,...,...,...,...,...,...,...
149,1995.262315,0.501187,0.398107,0.776527,0.444547,0.070988,0.001181,3.842983,1.9,3.4
150,1995.262315,0.794328,0.251189,0.332713,0.409091,0.003073,0.000597,9.213984,1.9,3.4
151,3162.277660,0.501187,0.630957,0.840547,0.351429,0.076566,0.001639,6.469137,1.9,3.4
152,3162.277660,0.794328,0.398107,0.415385,0.337500,0.003222,0.000747,16.812098,1.9,3.4


In [24]:
pess_df_y_degraded[:30]

,Q2,x,y,purity,stability,sigma_red,sigma_red_pdf_unc,sigma_stat,sigma_p2p,sigma_norm
0,1.258925,0.000200,0.630957,0.869573,0.473880,0.320509,0.031323,0.033080,1.900000,3.4
1,1.258925,0.000316,0.398107,0.778747,0.575700,0.290023,0.026500,0.026445,1.900000,3.4
2,1.258925,0.000501,0.251189,0.590162,0.564724,0.270261,0.016928,0.022658,1.900000,3.4
3,1.258925,0.000794,0.158489,0.382035,0.453197,0.254951,0.013751,0.020027,1.900000,3.4
4,1.258925,0.001259,0.100000,0.227740,0.300048,0.240664,0.012491,0.019144,1.900000,3.4
5,1.258925,0.001995,0.063096,0.152827,0.191763,0.224829,0.009176,0.019862,1.900000,3.4
6,1.258925,0.003162,0.039811,0.124728,0.125456,0.212840,0.008490,0.022618,2.386292,3.4
7,1.258925,0.005012,0.025119,0.114864,0.072073,0.206745,0.008751,0.029069,3.782018,3.4
8,1.258925,0.007943,0.015849,0.114361,0.047604,0.205227,0.007756,0.036368,5.994095,3.4
9,1.258925,0.012589,0.010000,0.132785,0.033178,0.205631,0.007700,0.047227,9.500000,3.4
